# Génération de Données Synthétiques avec GAN

Ce notebook implémente un Generative Adversarial Network (GAN) pour générer des données synthétiques d'utilisation d'applications.
Basé sur le tutoriel : [Synthetic Data Generation with Generative AI](https://amanxai.com/2024/10/14/synthetic-data-generation-with-generative-ai/)

## Structure du Projet
- Les fonctions principales sont dans `data_gen_utils.py`
- Les données d'origine sont dans `./data/screentime_analysis.csv`

In [11]:
# Import des bibliothèques nécessaires
import numpy as np
import pandas as pd

# Import de nos fonctions personnalisées
from data_gen_utils import (
    build_generator, build_discriminator, build_gan,
    train_gan, preprocess_data, generate_synthetic_data
)

## 1. Chargement et Préparation des Données

Nous chargeons les données d'utilisation d'applications et les préparons pour l'entraînement du GAN.

In [12]:
# Chargement des données
data = pd.read_csv('./data/screentime_analysis.csv')
print("Aperçu des données brutes :")
print(data.head())

# Prétraitement des données
normalized_data, scaler = preprocess_data(data)
print("\nDimensions des données normalisées:", normalized_data.shape)

Aperçu des données brutes :
         Date        App  Usage (minutes)  Notifications  Times Opened
0  2024-08-07  Instagram               81             24            57
1  2024-08-08  Instagram               90             30            53
2  2024-08-26  Instagram              112             33            17
3  2024-08-22  Instagram               82             11            38
4  2024-08-12  Instagram               59             47            16

Dimensions des données normalisées: (200, 3)


## 2. Configuration du GAN

Configuration et construction des modèles Generator et Discriminator.

In [13]:
# Paramètres du modèle
latent_dim = 100  # dimension de l'espace latent

# Construction des modèles
generator = build_generator(latent_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Affichage des architectures
print("Architecture du Generator :")
generator.summary()
print("\nArchitecture du Discriminator :")
discriminator.summary()

Architecture du Generator :


/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/synthetic-data-3D4-9DEH-py3.12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/synthetic-data-3D4-9DEH-py3.12/lib/python3.12/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 128)            │        12,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_6 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_7 (LeakyReLU)       │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_8 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 182,659 (713.51 KB)

...

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 512)            │         2,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_9 (LeakyReLU)       │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_10 (LeakyReLU)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_11 (LeakyReLU)      │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 166,401 (650.00 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 166,401 (650.00 KB)

## 3. Entraînement du Modèle

Entraînement du GAN sur nos données normalisées.

In [14]:
# Entraînement du GAN
train_gan(
    gan=gan,
    generator=generator,
    discriminator=discriminator,
    data=normalized_data,
    nb_epochs=100,  # Ajuster selon les besoins
    batch_size=128,
    latent_dim=latent_dim
)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


/home/briacleguillou/.var/app/com.visualstudio.code/cache/pypoetry/virtualenvs/synthetic-data-3D4-9DEH-py3.12/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:83: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0: D Loss: [0.6914638 0.7714844], G Loss: 0.6931794285774231
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━

## 4. Génération de Données Synthétiques

Utilisation du modèle entraîné pour générer de nouvelles données.

In [15]:
# Génération de nouvelles données
feature_names = [col for col in data.columns if col not in ['Date', 'App']]
generated_df = generate_synthetic_data(
    generator=generator,
    scaler=scaler,
    n_samples=1000,
    latent_dim=latent_dim,
    feature_names=feature_names
)

print("Aperçu des données générées :")
print(generated_df.head())

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Aperçu des données générées :
   Usage (minutes)  Notifications  Times Opened
0         1.000102     145.779831     98.980690
1       118.999969       0.233817      1.003303
2       118.976395       0.379667      1.972411
3       118.998901       0.066980      2.279521
4        27.851057       4.110766     98.968048
